# Notebook example of splitwise API usage

In [1]:
import pandas as pd

import splitwise_api_for_groceries as sag
import db_management as dbm 

/Users/saifwidyatmoko/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
secrets = dbm.secrets()
splitwise_api = secrets.get('API_KEY')

In [3]:
expenses_json = sag.splitwise_get_expenses_api_call(splitwise_api)

In [4]:
initial_df = sag.initial_groupings(expense=expenses_json)

In [5]:
print('initial dataset sourced after groupings is ' + str(len(initial_df)))

initial dataset sourced after groupings is 138


In [6]:
reduced_df = initial_df[['created_at','description','category',0,1,2,3]]

In [7]:
groceries_df = sag.source_groceries(reduced_df,desired_columns='category')
groceries_df = groceries_df.rename(columns={0:'zero',1:'one',2:'two',3:'three'})

/Users/saifwidyatmoko/Desktop/Python Projects/budgy/Budgy/splitwise_api_for_groceries.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_info[desired_columns] = table_info[desired_columns].apply(parse_json)
/Users/saifwidyatmoko/Desktop/Python Projects/budgy/Budgy/splitwise_api_for_groceries.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_info[desired_columns] = table_info[desired_columns].apply(lambda x: x['name'])


In [8]:
zero_df  = sag.normalize_dataset(groceries_df,desired_columns='zero')
one_df   = sag.normalize_dataset(groceries_df,desired_columns='one')
two_df   = sag.normalize_dataset(groceries_df,desired_columns='two')
three_df = sag.normalize_dataset(groceries_df,desired_columns='three')


In [9]:
mapped_groceries = groceries_df.reset_index().join([zero_df,one_df,two_df,three_df])
mapped_groceries = mapped_groceries.drop(columns=['zero','one','two','three'])

### Now that we have the mapped groceries, we can now map this against my personal splitwise expenses

In [10]:
mapped_groceries_reduced = mapped_groceries.drop(columns=['paid_share_zero',
                                     'paid_share_one',
                                     'paid_share_two',
                                     'paid_share_three'])


mapped_groceries_reduced.head()

,index,created_at,description,category,owed_share_zero,net_balance_zero,user.first_name_zero,owed_share_one,net_balance_one,user.first_name_one,owed_share_two,net_balance_two,user.first_name_two,owed_share_three,net_balance_three,user.first_name_three
0,0,2024-07-14T13:16:43Z,Grocc,Groceries,1.83,-1.83,Ned,1.82,1.83,Sang,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2024-07-13T15:03:20Z,Grocc,Groceries,1.02,-1.02,Saif,12.2,-12.2,Ned,0.57,-0.57,Kenny,6.86,13.79,Sang
2,3,2024-07-11T16:33:21Z,Groceries,Groceries,3.02,-3.02,Sang,3.5,8.57,Ned,5.55,-5.55,Saif,NaN,NaN,NaN
3,4,2024-07-10T15:03:15Z,Grocc,Groceries,5.23,-5.23,Saif,6.67,10.46,Sang,5.23,-5.23,Ned,NaN,NaN,NaN
4,6,2024-07-06T16:05:02Z,Grocc,Groceries,5.81,-5.81,Saif,1.1,-1.1,Kenny,2.0,-2.0,Ned,5.82,8.91,Sang


In [11]:
saif_stebonndale_df =  sag.saif_stebondale_expenses(mapped_groceries_reduced)
saif_stebonndale_df.head()

,index,created_at,description,category,owed_share_zero,net_balance_zero,user.first_name_zero,owed_share_one,net_balance_one,user.first_name_one,owed_share_two,net_balance_two,user.first_name_two,owed_share_three,net_balance_three,user.first_name_three,concat_user
1,2,2024-07-13T15:03:20Z,Grocc,Groceries,1.02,-1.02,Saif,12.2,-12.2,Ned,0.57,-0.57,Kenny,6.86,13.79,Sang,Saif_Ned_Kenny_Sang
2,3,2024-07-11T16:33:21Z,Groceries,Groceries,3.02,-3.02,Sang,3.5,8.57,Ned,5.55,-5.55,Saif,NaN,NaN,nan,Sang_Ned_Saif_nan
3,4,2024-07-10T15:03:15Z,Grocc,Groceries,5.23,-5.23,Saif,6.67,10.46,Sang,5.23,-5.23,Ned,NaN,NaN,nan,Saif_Sang_Ned_nan
4,6,2024-07-06T16:05:02Z,Grocc,Groceries,5.81,-5.81,Saif,1.1,-1.1,Kenny,2.0,-2.0,Ned,5.82,8.91,Sang,Saif_Kenny_Ned_Sang
5,7,2024-07-06T07:41:19Z,Grocc,Groceries,3.67,3.66,Saif,1.83,-1.83,Ned,1.83,-1.83,Sang,NaN,NaN,nan,Saif_Ned_Sang_nan


In [12]:
groceries_saif_dfs = []

# Iterate over multiple column indices and store the DataFrames in the list
for i in range(0, 4):
    u = sag.sourcing_saif_columns(saif_stebonndale_df, i)
    groceries_saif_dfs.append(u)

final_saif_df = pd.concat(groceries_saif_dfs)

In [13]:
final_saif_df['created_at'] = pd.to_datetime(final_saif_df['created_at']).dt.date 
final_saif_df = final_saif_df.drop(columns=['index'])
final_saif_df.head()

,created_at,description,category,owed_share,net_balance
0,2024-07-13,Grocc,Groceries,1.02,-1.02
1,2024-07-10,Grocc,Groceries,5.23,-5.23
2,2024-07-06,Grocc,Groceries,5.81,-5.81
3,2024-07-06,Grocc,Groceries,3.67,3.66
4,2024-07-04,Groceries,Groceries,3.82,-3.82


### Now we have the relevant data we can ingest this into our warehouse 

In [17]:
sql_server = dbm.generate_engine(secrets)

dbm.sql_ingestion(final_saif_df,named_table='groceries',engine_url=sql_server)

Successfully integrated


In [18]:
# test output in SQL 
dbm.create_query(engine=sql_server,query="""
                 SELECT * from groceries limit 40; """)

,created_at,description,category,owed_share,net_balance
0,2024-07-13,Grocc,Groceries,1.02,-1.02
1,2024-07-10,Grocc,Groceries,5.23,-5.23
2,2024-07-06,Grocc,Groceries,5.81,-5.81
3,2024-07-06,Grocc,Groceries,3.67,3.66
4,2024-07-04,Groceries,Groceries,3.82,-3.82
5,2024-07-02,Grocc,Groceries,2.02,-2.02
6,2024-06-27,Groceries,Groceries,1.4,-1.4
7,2024-06-26,Groceries,Groceries,2.12,-2.12
8,2024-06-24,Grocc,Groceries,3.36,-3.36
9,2024-06-20,米,Groceries,7.26,-7.26
